# Q2)

In [1]:

def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X

def ridge( A, d, la_array ):
    n = A.shape[1]
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        w = A.T@np.linalg.inv(A@A.T + each_lambda*np.eye(len(A@A.T)))@d
        X[:, i:i+1] = w
    return X


In [2]:
## Breast Cancer LASSO Exploration
## Prepare workspace
from scipy.io import loadmat
import numpy as np
X = loadmat("BreastCancer.mat")['X']
y = loadmat("BreastCancer.mat")['y']

##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,30],[31,60],[61,90],[91,120],[121,150],[151,179],[180,208],[209,237],[238,266],[267,295]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

# be sure to initiate the quantities you want to measure before looping
# through the various training, validation, and test partitions

avg_error_rate_lasso = 0
avg_sqd_error_ridge = 0
avg_error_rate_ridge = 0
avg_sqd_error_lasso = 0
lam_vals = np.logspace(-8,np.log10(20),100)

# Loop over various cases
for j in range(cases):
    print("Iteration: ", j+1)
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(295))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = X[v1_ind,:]
    bv1 = y[v1_ind]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = X[v2_ind,:]
    bv2 = y[v2_ind]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = X[trn_ind,:]
    bt = y[trn_ind]
    
    print(len(v1_ind), len(v2_ind), len(trn_ind))
    
# Use training data to learn classifier weights
    wmat_lasso = ista_solve_hot(At,bt,lam_vals)
    wmat_ridge = ridge(At,bt,lam_vals)
    lam_lasso = None
    w_opt_lasso = None
    min_error_lasso = -1
    lam_ridge = None
    w_opt_ridge= None
    min_error_ridge = -1
    
# Find best lambda value using the first validation set, then evaluate everything from there
    for i in range(len(wmat_lasso[0])):
        w_lasso = wmat_lasso[:, i:i+1]
        d_hat_lasso = np.sign(Av1@w_lasso)
        error_vec_lasso = [0 if m[1]==m[0] else 1 for m in np.hstack((d_hat_lasso, bv1))]
        error_rate_lasso = sum(error_vec_lasso)/len(error_vec_lasso)
        if min_error_lasso == -1 or error_rate_lasso<min_error_lasso:
            min_error_lasso = error_rate_lasso
            lam_lasso = lam_vals[i]
            w_opt_lasso = w_lasso
        w_ridge = wmat_ridge[:, i:i+1]
        d_hat_ridge = np.sign(Av1@w_ridge)
        error_vec_ridge = [0 if m[1]==m[0] else 1 for m in np.hstack((d_hat_ridge, bv1))]
        error_rate_ridge = sum(error_vec_ridge)/len(error_vec_ridge)
        if min_error_ridge == -1 or error_rate_ridge<min_error_ridge:
            min_error_ridge = error_rate_ridge
            lam_ridge = lam_vals[i]
            w_opt_ridge = w_ridge

# perform on second validation set, and accumulate performance metrics over all cases and their respective splittings
    print("Best lambda in LASSO: ", lam_lasso)
    print("Best lambda in ridge regression: ", lam_ridge)
    d_hat_lasso = np.sign(Av2@w_opt_lasso)
    error_vec_lasso = [0 if m[1]==m[0] else 1 for m in np.hstack((d_hat_lasso,bv2))]
    error_rate_lasso = sum(error_vec_lasso)/len(error_vec_lasso)
    avg_error_rate_lasso += error_rate_lasso
    squared_error_lasso = np.linalg.norm(d_hat_lasso-bv2)**2
    avg_sqd_error_lasso += squared_error_lasso
    print("Error rate of lasso: ", error_rate_lasso)
    print("Squared error of lasso: ", squared_error_lasso)
    print("avg error rate of lasso: ",  avg_error_rate_lasso)
    d_hat_ridge = np.sign(Av2@w_opt_ridge)
    error_vec_ridge = [0 if m[1]==m[0] else 1 for m in np.hstack((d_hat_ridge,bv2))]
    error_rate_ridge = sum(error_vec_ridge)/len(error_vec_ridge)
    avg_error_rate_ridge += error_rate_ridge
    squared_error_ridge = np.linalg.norm(d_hat_ridge-bv2)**2
    avg_sqd_error_ridge += squared_error_ridge
    print("Error rate of ridge: ", error_rate_ridge)
    print("Squared error of ridge: ", squared_error_ridge)
    print("avg error rate of ridge: ",  avg_error_rate_ridge)
    print("\n\n")
avg_error_rate_ridge /= cases
avg_sqd_error_ridge /= cases
avg_error_rate_lasso /= cases
avg_sqd_error_lasso /= cases
print("Average error rate of lasso: ", avg_error_rate_lasso)
print("Average squared error of lasso: ", avg_sqd_error_lasso)
print("Average error rate of ridge: ", avg_error_rate_ridge)
print("Average squared error of ridge: ", avg_sqd_error_ridge)


Iteration:  1
30 30 235
Best lambda in LASSO:  3.5434932692979846
Best lambda in ridge regression:  1e-08
Error rate of lasso:  0.43333333333333335
Squared error of lasso:  51.99999999999999
avg error rate of lasso:  0.43333333333333335
Error rate of ridge:  0.43333333333333335
Squared error of ridge:  51.99999999999999
avg error rate of ridge:  0.43333333333333335



Iteration:  2
30 30 235
Best lambda in LASSO:  16.109430878355187
Best lambda in ridge regression:  1e-08
Error rate of lasso:  0.26666666666666666
Squared error of lasso:  32.00000000000001
avg error rate of lasso:  0.7
Error rate of ridge:  0.23333333333333334
Squared error of ridge:  28.000000000000004
avg error rate of ridge:  0.6666666666666667



Iteration:  3
30 30 235
Best lambda in LASSO:  3.5434932692979846
Best lambda in ridge regression:  1e-08
Error rate of lasso:  0.4666666666666667
Squared error of lasso:  56.0
avg error rate of lasso:  1.1666666666666665
Error rate of ridge:  0.43333333333333335
Squared er